## SVD avec surprise : méthode de Simon Funk avec descente de gradient permettant de contourner la sparsité

In [5]:
import pandas as pd
from surprise import Dataset, Reader

ratings = pd.read_csv(r"C:\Users\nolle\OneDrive\Documents\Université\M2\Mémoire\Base de données traitées\All.csv")
ratings = ratings[["userId", "title_clean_x", "rating"]]

In [8]:
# Définir un Reader avec la plage des notes
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))

# Charger le dataset depuis le DataFrame
data = Dataset.load_from_df(ratings[['userId', 'title_clean_x', 'rating']], reader)

##  I. Création de la fonction permettant d'obtenir les 10 recommandations les plus pertinentes

In [11]:
def recommend_movies_for_user(user_id, ratings_df, algo, n=10):
    """
    Recommande les n meilleurs films non-vus par l'utilisateur à partir d'un modèle entraîné.
    
    Args:
        user_id (int): ID de l'utilisateur
        ratings_df (pd.DataFrame): DataFrame contenant userId, title_clean_x, rating
        algo (surprise.AlgoBase): Modèle entraîné (par ex. SVD)
        n (int): Nombre de recommandations à retourner

    Returns:
        List of tuples: [(title, predicted_rating), ...]
    """
    all_titles = ratings_df['title_clean_x'].unique()
    seen_titles = ratings_df[ratings_df['userId'] == user_id]['title_clean_x'].unique()
    unseen_titles = [title for title in all_titles if title not in seen_titles]

    predictions = [algo.predict(user_id, title) for title in unseen_titles]
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    return [(pred.iid, round(pred.est, 2)) for pred in predictions[:n]]

In [14]:
recommendations = recommend_movies_for_user(5, ratings, algo, n=10)
for title, score in recommendations:
    print(f"{title} — {score}")

Pee-wee's Big Adventure (1985) — 4.54
M (1931) — 4.53
Double Indemnity (1944) — 4.49
Who's Afraid of Virginia Woolf? (1966) — 4.46
Modern Times (1936) — 4.45
This Is Spinal Tap (1984) — 4.43
Ran (1985) — 4.38
Requiem for a Dream (2000) — 4.37
Sanjuro (1962) — 4.33
Roger & Me (1989) — 4.32


## II. Qualité des prédictions

In [16]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm


true_ratings = []
pred_ratings = []

reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))
user_ids = ratings['userId'].unique()

for user_id in tqdm(user_ids, desc="Validation réaliste par utilisateur"):
    # Toutes ses notes
    user_data = ratings[ratings['userId'] == user_id]

    # Skip s'il n'a pas assez de notes
    if len(user_data) < 5:
        continue

    # Shuffle et split
    user_data = user_data.sample(frac=1, random_state=42)
    split_idx = int(0.8 * len(user_data))
    user_train = user_data.iloc[:split_idx]
    user_test = user_data.iloc[split_idx:]

    # Données des autres utilisateurs
    others_data = ratings[ratings['userId'] != user_id]

    # Fusion pour entraînement : 80 % de l'utilisateur + tous les autres
    train_df = pd.concat([user_train, others_data])

    # Entraînement du modèle SVD collaboratif
    data = Dataset.load_from_df(train_df[['userId', 'title_clean_x', 'rating']], reader)
    trainset = data.build_full_trainset()
    algo = SVD()
    algo.fit(trainset)

    # Prédictions sur les 20 % cachés
    for _, row in user_test.iterrows():
        pred = algo.predict(uid=row['userId'], iid=row['title_clean_x']).est
        true_ratings.append(row['rating'])
        pred_ratings.append(pred)

Validation réaliste par utilisateur:  33%|███▎      | 2000/6040 [45:32:01<91:58:42, 81.96s/it]      


KeyboardInterrupt: 

In [2]:
# Interruption de la requête à 2000 utilisateurs testés sur 6040 car requête trop gourmande (46 heures pour 2000 utilisateurs)

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(true_ratings, pred_ratings, squared=False)
mae = mean_absolute_error(true_ratings, pred_ratings)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

RMSE: 0.8465
MAE: 0.6641
